In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import random
import json

from data_utils import MLDataset, collate_fn
from modeling_challenge import Seq2SeqModel

In [ ]:
assert torch.cuda.is_available()

# Use 0th GPU for training
torch.cuda.set_device(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# fix random seed to increase reproducibility
# NOTE: Do not modify here!
NUM_CLASSES = 26 + 2  # 26 alphabets + 1 padding index + 1 <s> token (start token)

random_seed = 7
torch.manual_seed(random_seed)
os.environ["PYTHONHASHSEED"] = str(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)
torch.cuda.manual_seed(random_seed)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
# %env CUBLAS_WORKSPACE_CONFIG=:16:8


def seed_worker(worker_seed):
    np.random.seed(worker_seed)
    random.seed(worker_seed)


num_workers = 8

In [ ]:
# Select augmentations to apply
augmentations = [
    "random_translate",
    # "color_jitter",
    # "random_rotation",
    # "gaussian_blur"
]

In [ ]:
# NOTE: modify path and batch size for your setting
# NOTE: you can apply custom preprocessing to the training data

BATCH_SIZE = 128

train_ds = MLDataset("data_final/imgs/train", "data_final/labels/train.json")
valid_ds = MLDataset(
    "data_final/imgs/valid_normal", "data_final/labels/valid_normal.json"
)
challenge_ds = MLDataset(
    "data_final/imgs/valid_challenge", "data_final/labels/valid_challenge.json"
)

train_dl = DataLoader(
    train_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True
)
valid_dl = DataLoader(
    valid_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False
)
challenge_dl = DataLoader(
    challenge_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False
)

In [ ]:
# You can add or modify your Seq2SeqModel's hyperparameter (keys and values)
kwargs = {
    'hidden_dim': 256,       # Hidden dimension size for RNN
    'nhead': 4,              # Number of attention heads in the Transformer
    'dec_layers': 4,         # Number of layers in the Transformer decoder
    'dim_feedforward': 1024, # Dimension of feedforward layers in the Transformer
    'dropout': 0.2,          # Dropout rate for the Transformer
    'enc_layers': 4,         # Number of RNN layers in the encoder
    'rnn_dropout': 0.2,      # Dropout rate for the RNN in the encoder
    'max_length': 11,        # Maximum length of the sequences
    'cnn_settings': {        # Settings for the CustomCNN
        'block1_dim': 32,
        'block2_dim': 64,
        'block3_dim': 128,
        'fc_dim': 256,
        'model_type': 'VGG'  # Type of CNN ('VGG' or 'ResNet')
    },
}

In [ ]:
model = Seq2SeqModel(num_classes=NUM_CLASSES, **kwargs).to(device)
print(model)
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
model_optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(model_optim, step_size=10, gamma=0.5)
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################